<a href="https://colab.research.google.com/github/monipip3/women_in_data_datathon/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

+ Read in Paid leave reviews
+ Join as single dataframe
+ add sentiment to each review
+ join to other dataset
+ get the avg sentiment post change
+ get the % change of sentiment of those companies that have dates on paid leave change policy

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 163 kB 53.1 MB/s 
     |████████████████████████████████| 6.6 MB 38.3 MB/s 


In [3]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.2 MB/s 


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")

model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-emotion")

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [5]:
import joblib
import pandas as pd
import os
from textblob import TextBlob


In [6]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Datathon/paid_leave_reviews

Mounted at /content/drive
/content/drive/MyDrive/Datathon/paid_leave_reviews


In [7]:
%cd ../

/content/drive/MyDrive/Datathon


In [8]:
paid_leave_files = os.listdir('paid_leave_reviews')

In [9]:
len(paid_leave_files)

49

In [10]:
reviews_dataframe = joblib.load(f'./paid_leave_reviews/{paid_leave_files[0]}')
for company in paid_leave_files[1:]:
  temp_dataframe = joblib.load(f'./paid_leave_reviews/{company}')
  reviews_dataframe = reviews_dataframe.append(temp_dataframe,ignore_index=True)

In [11]:
reviews_dataframe.shape

(4834, 5)

In [12]:
def get_emotion(text):
  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')

  output = model.generate(input_ids=input_ids,
               max_length=2)
  
  dec = [tokenizer.decode(ids) for ids in output]
  label = dec[0]
  return(label)

In [13]:
reviews_dataframe['emotion'] = reviews_dataframe.Summary.apply(get_emotion)

In [14]:
blobs = [TextBlob(text) for text in reviews_dataframe.Summary.values]

In [16]:
polarities = [sentence.sentiment.polarity for sentence in blobs]
#for sentence in blob.sentences:
   # print(sentence.sentiment.polarity)

In [17]:
reviews_dataframe['polarity'] = polarities

In [18]:
reviews_dataframe[['emotion','polarity']].groupby('emotion').describe()

polarity                                                \
                  count      mean       std       min       25%   50%   
emotion                                                                 
<pad> anger      1023.0  0.034196  0.174674 -1.000000  0.000000  0.00   
<pad> fear        530.0  0.025175  0.167545 -1.000000  0.000000  0.00   
<pad> joy        2964.0  0.330034  0.340735 -0.714286  0.000000  0.25   
<pad> love        109.0  0.225566  0.206558 -0.062500  0.075000  0.20   
<pad> sadness     178.0 -0.034571  0.225921 -1.000000 -0.074583  0.00   
<pad> surprise     30.0  0.527764  0.230490  0.000000  0.362500  0.60   

                                  
                     75%     max  
emotion                           
<pad> anger     0.051042  0.8000  
<pad> fear      0.000000  1.0000  
<pad> joy       0.600000  1.0000  
<pad> love      0.275000  0.8125  
<pad> sadness   0.019196  0.5000  
<pad> surprise  0.600000  1.0000

In [19]:
reviews_dataframe['Date'] = pd.to_datetime(reviews_dataframe.Date)

In [ ]:
reviews_dataframe.dtypes

Summary                  object
Date             datetime64[ns]
JobTitle                 object
OverallRating           float64
company                  object
emotion                  object
polarity                float64
dtype: object

In [20]:
joblib.dump(reviews_dataframe,'reviews_all_companies.pkl')

['reviews_all_companies.pkl']

In [21]:
reviews_dataframe = joblib.load('reviews_all_companies.pkl')

In [22]:
reviews_dataframe.shape

(4834, 7)

In [23]:
reviews_dataframe.to_csv('reviews_all_companies.csv',index=False)

In [24]:
large_companies = pd.read_excel('large_companies.xlsx',sheet_name='main_data')

In [25]:
large_companies.head()

,Company,num_employees,sector,avg_paid_leave_2022,Revenue(B)_qtr_June22,median_retention_years_2022,Paid Leave_amt__maternity_after_change_weeks,Paid Leave_amt_paternity_after_change_weeks,polarity_change_pct_after_change,polarity_after_change,polarity_before_change,Date Changed,Press release / Links,Unnamed: 13
0,Walmart,2300000.0,Retail,3.9,572.800,4.2,10.0,6.0,NaN,NaN,NaN,2018-01-01,https://corporate.walmart.com/newsroom/2018/01...,NaN
1,Amazon,1608000.0,Internet & Web Services,4.1,121.240,0.8,20.0,6.0,NaN,NaN,NaN,2015-11-01,https://www.aboutamazon.com/news/workplace/wha...,NaN
2,Apple,154000.0,Computer Hardware Development,4.6,82.959,0.3,20.0,6.0,NaN,NaN,NaN,2019-11-01,https://www.imore.com/apple-expands-parental-l...,NaN
3,CVS Health,300000.0,Health Care Services & Hospitals,3.2,80.636,4.5,10.0,4.0,NaN,NaN,NaN,2018-04-01,https://www.theshelbyreport.com/2018/02/21/cvs...,NaN
4,UnitedHealth Group,350000.0,Health Care Services & Hospitals,3.6,80.332,6.7,4.0,4.0,NaN,NaN,NaN,2018-01-01,https://fairygodboss.com/parental-leave/united...,NaN


In [26]:
large_companies2 = large_companies[~large_companies['Date Changed'].isna()]

In [27]:
large_companies2.shape

(23, 14)

In [28]:
reviews_dataframe = reviews_dataframe.merge(large_companies2[['Company','Date Changed']],left_on="company",right_on="Company")

In [36]:
reviews_dataframe.isna().sum()

Summary               0
Date                  0
JobTitle              0
OverallRating         0
company               0
emotion               0
polarity              0
Company               0
Date Changed          0
post_policy_change    0
dtype: int64

In [29]:
reviews_dataframe.columns

Index(['Summary', 'Date', 'JobTitle', 'OverallRating', 'company', 'emotion',
       'polarity', 'Company', 'Date Changed'],
      dtype='object')

In [30]:
reviews_dataframe.dtypes

Summary                  object
Date             datetime64[ns]
JobTitle                 object
OverallRating           float64
company                  object
emotion                  object
polarity                float64
Company                  object
Date Changed     datetime64[ns]
dtype: object

In [31]:
def post_policy_change_flag(x,y):
  if x >= y:
    return(True)
  else:
    return(False)

In [32]:
reviews_dataframe['post_policy_change'] = reviews_dataframe.apply(lambda x: post_policy_change_flag(x['Date'], x['Date Changed']), axis=1)

In [33]:
reviews_dataframe[['post_policy_change','polarity']].groupby('post_policy_change').describe()

polarity                                                   
                      count      mean       std   min  25%  50%       75%  max
post_policy_change                                                            
False                1598.0  0.225657  0.326660 -0.91  0.0  0.1  0.401786  1.0
True                 1427.0  0.232272  0.331688 -1.00  0.0  0.1  0.455000  1.0

In [37]:
reviews_dataframe[['company','post_policy_change','polarity']].groupby(['company','post_policy_change']).describe()

polarity                      \
                                           count      mean       std   
company              post_policy_change                                
Alphabet             False                 112.0  0.267088  0.358730   
                     True                   29.0  0.238945  0.411308   
Amazon               False                  10.0  0.180188  0.435360   
                     True                  275.0  0.231063  0.333818   
Apple                False                  92.0  0.222369  0.322810   
                     True                   71.0  0.276076  0.326541   
Bank of America      False                 174.0  0.286758  0.372377   
                     True                  259.0  0.189678  0.330917   
Berkshire Hathaway   True                    2.0  0.100000  0.141421   
CVS Health           False                  51.0  0.063129  0.212274   
                     True                   43.0  0.171486  0.250935   
Citigroup            False                 163.0  0.196412  0.293993   
                     True                   60.0  0.245181  0.366949   
Exxon Mobil          False                  18.0  0.275512  0.269533   
                     True                   20.0  0.126947  0.296504   
Ford Motor           False                   6.0  0.100000  0.352136   
                     True                   24.0  0.189166  0.333815   
Home Depot           False                  45.0  0.183615  0.326780   
                     True                   61.0  0.285418  0.330881   
IBM                  False                  92.0  0.179958  0.281874   
                     True                  138.0  0.306239  0.362884   
Intel                False                 141.0  0.234495  0.311529   
                     True                   67.0  0.242926  0.343581   
Johnson & Johnson    False                  40.0  0.334473  0.351675   
                     True                   32.0  0.333080  0.337570   
Marathon Petroleum   False                   8.0  0.068056  0.401065   
                     True                    8.0 -0.008958  0.016588   
McKesson             False                  12.0  0.017940  0.285572   
                     True                   25.0  0.109561  0.292080   
Microsoft            False                 287.0  0.284559  0.366762   
                     True                   20.0  0.256944  0.392907   
Pfizer               False                  24.0  0.161392  0.271287   
                     True                   19.0  0.420941  0.385501   
Procter & Gamble     False                  29.0  0.252905  0.285860   
                     True                   15.0  0.153611  0.231588   
State Farm Insurance False                  28.0  0.046181  0.257032   
                     True                   18.0  0.202778  0.306106   
Target               False                 103.0  0.194728  0.294290   
                     True                   39.0  0.137578  0.247522   
UnitedHealth Group   False                  41.0  0.130533  0.218931   
                     True                   38.0  0.133727  0.270404   
Walmart              False                 122.0  0.214244  0.273082   
                     True                  164.0  0.268764  0.305090   

                                                                       \
                                              min       25%       50%   
company              post_policy_change                                 
Alphabet             False              -0.291667  0.000000  0.145833   
                     True               -1.000000  0.000000  0.150000   
Amazon               False              -0.500000 -0.012784  0.035714   
                     True               -0.700000  0.000000  0.125000   
Apple                False              -0.714286  0.000000  0.122619   
                     True               -0.407143  0.000000  0.200000   
Bank of America      False              -0.700000  0.000000  0.185417 